# The group task

- Loading packages

In [1]:
import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
import pyreadstat
# To display the graphs here
%matplotlib inline

- Importing the data

In [25]:
health = pd.read_sas("/Users/nathankhadaroo/Desktop/task/Group/london ward data health.sas7bdat", format='sas7bdat', encoding='latin1')
environment = pd.read_csv("/Users/nathankhadaroo/Desktop/task/Group/London ward data environment.csv")
district_codes = pd.read_csv("/Users/nathankhadaroo/Desktop/task/Group/LondonDistrictcodes.csv")
demo = pd.read_table("/Users/nathankhadaroo/Desktop/task/Group/London ward data demographics.dat")
socio  = pd.read_csv("/Users/nathankhadaroo/Desktop/task/Group/lsd.csv")

In [63]:
district_codes.head()

,District,Districtcode
0,Barking and Dagenham,00AB
1,Barnet,00AC
2,Bexley,00AD
3,Brent,00AE
4,Bromley,00AF


In [27]:
demo.head()

,Wardname,Children,Greaterthan65,nonwhite,NotBorninUK,NotEnglishspeaking
0,Hackney - Queensbridge,17.542063,8.2,44.7,38.3,11.8
1,Hammersmith & Fulham - S&s End,17.915361,8.4,30.6,39.0,11.8
2,Barking & Dagenham - River,26.851598,10.1,38.4,30.9,9.6
3,Tower Hamlets - Bethnal Green North,18.555872,7.0,49.4,38.7,15.0
4,Merton - Abbey,15.731861,8.5,26.6,38.3,12.5


In [28]:
environment.head()

,Wardcode,Population2011Census,Crimerate,Openspace
0,00ANGQ,11201,117.7,0.0
1,00ANGA,11518,114.0,0.3
2,00ADGN,10800,44.2,0.7
3,00BEGH,12321,65.3,0.7
4,00BCFZ,12609,64.3,1.3


In [29]:
socio.head()

,Wardcode,hhSocialRented,JobSeekers,Noqual,Carsperhousehold
0,00ABFX,26.7,8.7,16.4,0.5
1,00ABFY,36.8,10.2,31.2,0.8
2,00ABFZ,29.4,10.2,28.0,0.9
3,00ABGB,20.0,8.8,29.9,1.1
4,00ABGC,37.4,10.2,28.9,0.8


In [30]:
health.head()

,Wardname,Population2011Census,GeneralFertilityRate,Malelifeexpectancy,Femalelifeexpectancy
0,Bromley - Darwin,5110.0,63.8,81.2,82.4
1,Kensington and Chelsea - Royal Hospital,7252.0,52.3,80.5,89.6
2,Hillingdon - Harefield,7399.0,55.8,78.3,82.3
3,Hammersmith and Fulham - Palace Riverside,7483.0,40.0,80.9,89.0
4,Kensington and Chelsea - Pembridge,7659.0,40.4,82.1,85.7


In [55]:
merge1 = pd.merge(health, demo)
merge1['Population2011Census'] = merge1['Population2011Census'].astype(int)

In [56]:
merge1.head()

,Wardname,Population2011Census,GeneralFertilityRate,Malelifeexpectancy,Femalelifeexpectancy,Children,Greaterthan65,nonwhite,NotBorninUK,NotEnglishspeaking
0,Bromley - Darwin,5110,63.8,81.2,82.4,18.317025,21.9,4.8,6.1,0.4
1,Hillingdon - Harefield,7399,55.8,78.3,82.3,19.245844,17.0,12.5,13.3,3.1
2,Westminster - Tachbrook,8158,33.3,82.2,83.1,10.750184,15.4,21.8,41.1,15.1
3,Merton - Village,8491,49.7,82.6,86.3,19.102579,19.3,17.1,34.0,8.3
4,Merton - Hillside,8977,60.0,81.0,91.7,13.824217,13.7,21.7,41.6,11.3


In [57]:
merge1.to_csv("/Users/nathankhadaroo/Desktop/task/Group/lol.csv")

In [58]:
merge2 = pd.merge(environment, socio)

In [64]:
merge2.head()

,Wardcode,Population2011Census,Crimerate,Openspace,hhSocialRented,JobSeekers,Noqual,Carsperhousehold
0,00ANGQ,11201,117.7,0.0,23.7,3.6,9.4,0.6
1,00ANGA,11518,114.0,0.3,24.9,4.5,9.3,0.5
2,00ADGN,10800,44.2,0.7,1.1,2.9,22.1,1.3
3,00BEGH,12321,65.3,0.7,20.3,4.6,12.3,0.7
4,00BCFZ,12609,64.3,1.3,4.7,4.2,17.2,1.2


In [67]:
merged = pd.merge(merge1, merge2)

In [68]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 556 entries, 0 to 555
Data columns (total 17 columns):
Wardname                556 non-null object
Population2011Census    556 non-null int64
GeneralFertilityRate    556 non-null float64
Malelifeexpectancy      556 non-null float64
Femalelifeexpectancy    556 non-null float64
Children                556 non-null float64
Greaterthan65           556 non-null float64
nonwhite                556 non-null float64
NotBorninUK             556 non-null float64
NotEnglishspeaking      556 non-null float64
Wardcode                556 non-null object
Crimerate               556 non-null float64
Openspace               556 non-null float64
hhSocialRented          556 non-null float64
JobSeekers              556 non-null float64
Noqual                  556 non-null float64
Carsperhousehold        556 non-null float64
dtypes: float64(14), int64(1), object(2)
memory usage: 78.2+ KB


In [69]:
merged.head()

,Wardname,Population2011Census,GeneralFertilityRate,Malelifeexpectancy,Femalelifeexpectancy,Children,Greaterthan65,nonwhite,NotBorninUK,NotEnglishspeaking,Wardcode,Crimerate,Openspace,hhSocialRented,JobSeekers,Noqual,Carsperhousehold
0,Bromley - Darwin,5110,63.8,81.2,82.4,18.317025,21.9,4.8,6.1,0.4,00AFGQ,58.6,88.8,5.5,1.6,21.9,1.7
1,Hillingdon - Harefield,7399,55.8,78.3,82.3,19.245844,17.0,12.5,13.3,3.1,00ASGN,67.4,85.5,24.0,3.2,23.8,1.4
2,Westminster - Tachbrook,8158,33.3,82.2,83.1,10.750184,15.4,21.8,41.1,15.1,00BKGR,51.1,32.4,29.3,2.7,12.5,0.4
3,Merton - Village,8491,49.7,82.6,86.3,19.102579,19.3,17.1,34.0,8.3,00BAGQ,60.3,62.2,5.6,0.9,6.8,1.4
4,Merton - Hillside,8977,60.0,81.0,91.7,13.824217,13.7,21.7,41.6,11.3,00BAGE,52.0,5.7,9.9,1.2,6.4,0.8


# Preparing the machine learning model